# Kaggle House Price Prob.



2018.10.20.Sat. By Taeu

for Google Machine learning study

*******************************************************

# Content
## 1. Data Skimming

    1-1. Data collect
    1-2. Data load
    1-3. Data Skimming ( to excel _ )


## 2. Data Preprocessing

    2-1. Fill NAs
    2-2. Drop
    2-3. Editing ( Adding, Editing, Binning etc..)
    
    
## 3. Model & Evaluation

    3-1 pipeline
    3-2 Model Selection & Eval
    3-3 Result - Submission
    
    
*******************************************************

In [1]:
# import 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline, make_pipeline
from scipy.stats import skew
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import Imputer

from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import ElasticNet, SGDRegressor, BayesianRidge
from sklearn.kernel_ridge import KernelRidge
#from xgboost import XGBRegressor
# if you want to use XGBoost
'''
# install xgboost
http://cleancode-ws.tistory.com/79
# understanding xgboost
https://www.slideshare.net/freepsw/boosting-bagging-vs-boosting
'''

'\n# install xgboost\nhttp://cleancode-ws.tistory.com/79\n# understanding xgboost\nhttps://www.slideshare.net/freepsw/boosting-bagging-vs-boosting\n'

## 1. Data Skimming

In [2]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
full = pd.concat([train,test],ignore_index=True)
cols = train.columns
nulls = full.isnull().sum()
nulls = nulls[nulls>0]
nulls_index = nulls.index
datacols = []
datanull = []
datatype = []
datavalue = []

for col in cols:
    datacols += [col]
    datatype += [train[col].dtype]
    datavalue+= [train[col][0]]
    if col in nulls_index :
        datanull += [nulls[col]]
    else :
        datanull += [0]
        
idd = list(range(cols.size))

## 2. Data Preprocessing

### 2-1. Missing Value

In [3]:
full.groupby(['Neighborhood'])[['LotFrontage']].agg(['mean','median','count'])

full['LotAreaCut']= pd.qcut(full.LotArea,10)

full['LotFrontage'] = full.groupby(['LotAreaCut','Neighborhood'])['LotFrontage'].transform(lambda x : x.fillna(x.median()))

# 나머지 변수들은 다 0 이나 None 값으로
# nulls_index = nulls.index

for col in nulls_index:
    na_value = 0
    if full[col].dtype == object :
        na_value = "None"
    full[col].fillna(na_value, inplace = True)

### 2-2. drop 

In [4]:
drop_index = ['SalePrice','Id']
for var in drop_index:
    full.drop([var],axis=1,inplace = True)

### 2-3. Feature Engineering


#### (1) type 변경

In [5]:
full_col = full.columns
for var in full_col:
    if full[var].dtype == object :
        full[var] = full[var].astype(str)
        
time = ['GarageYrBlt','YearBuilt','YearRemodAdd','MoSold','YrSold']
categorical = ['MSSubClass','OverallQual','OverallCond',]
strtype = time + categorical
for var in strtype:
    full[var] = full[var].astype(str)
    
full.drop('LotAreaCut',axis=1,inplace=True)
full_col = full.columns

#### (2) scale 조정

In [6]:
# numeric 변수들 모음
numeric_col = []
for var in full_col:
    if type(full[var][0]) != str:
        numeric_col += [var]
# http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
# ~.fit(data), ~.fit_transform()

minmax = MinMaxScaler()
standard = StandardScaler()
robust = RobustScaler()

# 각 scale을 따로 적용하기 위해서 full의 copy version을 만든다.
full2 = full.copy()
full3 = full.copy()
full4 = full.copy()

#full1 은 no scale
full2[numeric_col] = minmax.fit_transform(full2[numeric_col])
full3[numeric_col] = standard.fit_transform(full3[numeric_col])
full4[numeric_col] = robust.fit_transform(full4[numeric_col])


#### (3) categorical 변수 조정

In [7]:
# map 함수 설정
def map_values(full):
    full["oMSSubClass"] = full.MSSubClass.map({'180':1, 
                                        '30':2, '45':2, 
                                        '190':3, '50':3, '90':3, 
                                        '85':4, '40':4, '160':4, 
                                        '70':5, '20':5, '75':5, '80':5, '150':5,
                                        '120': 6, '60':6})
    
    full["oMSZoning"] = full.MSZoning.map({'C (all)':1, 'RH':2, 'RM':2, 'RL':3, 'FV':4})
    
    full["oNeighborhood"] = full.Neighborhood.map({'MeadowV':1,
                                               'IDOTRR':2, 'BrDale':2,
                                               'OldTown':3, 'Edwards':3, 'BrkSide':3,
                                               'Sawyer':4, 'Blueste':4, 'SWISU':4, 'NAmes':4,
                                               'NPkVill':5, 'Mitchel':5,
                                               'SawyerW':6, 'Gilbert':6, 'NWAmes':6,
                                               'Blmngtn':7, 'CollgCr':7, 'ClearCr':7, 'Crawfor':7,
                                               'Veenker':8, 'Somerst':8, 'Timber':8,
                                               'StoneBr':9,
                                               'NoRidge':10, 'NridgHt':10})
    
    full["oCondition1"] = full.Condition1.map({'Artery':1,
                                           'Feedr':2, 'RRAe':2,
                                           'Norm':3, 'RRAn':3,
                                           'PosN':4, 'RRNe':4,
                                           'PosA':5 ,'RRNn':5})
    
    full["oBldgType"] = full.BldgType.map({'2fmCon':1, 'Duplex':1, 'Twnhs':1, '1Fam':2, 'TwnhsE':2})
    
    full["oHouseStyle"] = full.HouseStyle.map({'1.5Unf':1, 
                                           '1.5Fin':2, '2.5Unf':2, 'SFoyer':2, 
                                           '1Story':3, 'SLvl':3,
                                           '2Story':4, '2.5Fin':4})
    
    full["oExterior1st"] = full.Exterior1st.map({'BrkComm':1,
                                             'AsphShn':2, 'CBlock':2, 'AsbShng':2,
                                             'WdShing':3, 'Wd Sdng':3, 'MetalSd':3, 'Stucco':3, 'HdBoard':3,
                                             'BrkFace':4, 'Plywood':4,
                                             'VinylSd':5,
                                             'CemntBd':6,
                                             'Stone':7, 'ImStucc':7})
    
    full["oMasVnrType"] = full.MasVnrType.map({'BrkCmn':1, 'None':1, 'BrkFace':2, 'Stone':3})
    
    full["oExterQual"] = full.ExterQual.map({'Fa':1, 'TA':2, 'Gd':3, 'Ex':4})
    
    full["oFoundation"] = full.Foundation.map({'Slab':1, 
                                           'BrkTil':2, 'CBlock':2, 'Stone':2,
                                           'Wood':3, 'PConc':4})
    
    full["oBsmtQual"] = full.BsmtQual.map({'Fa':2, 'None':1, 'TA':3, 'Gd':4, 'Ex':5})
    
    full["oBsmtExposure"] = full.BsmtExposure.map({'None':1, 'No':2, 'Av':3, 'Mn':3, 'Gd':4})
    
    full["oHeating"] = full.Heating.map({'Floor':1, 'Grav':1, 'Wall':2, 'OthW':3, 'GasW':4, 'GasA':5})
    
    full["oHeatingQC"] = full.HeatingQC.map({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
    
    full["oKitchenQual"] = full.KitchenQual.map({'Fa':1, 'TA':2, 'Gd':3, 'Ex':4})
    
    full["oFunctional"] = full.Functional.map({'Maj2':1, 'Maj1':2, 'Min1':2, 'Min2':2, 'Mod':2, 'Sev':2, 'Typ':3})
    
    full["oFireplaceQu"] = full.FireplaceQu.map({'None':1, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
    
    full["oGarageType"] = full.GarageType.map({'CarPort':1, 'None':1,
                                           'Detchd':2,
                                           '2Types':3, 'Basment':3,
                                           'Attchd':4, 'BuiltIn':5})
    
    full["oGarageFinish"] = full.GarageFinish.map({'None':1, 'Unf':2, 'RFn':3, 'Fin':4})
    
    full["oPavedDrive"] = full.PavedDrive.map({'N':1, 'P':2, 'Y':3})
    
    full["oSaleType"] = full.SaleType.map({'COD':1, 'ConLD':1, 'ConLI':1, 'ConLw':1, 'Oth':1, 'WD':1,
                                       'CWD':2, 'Con':3, 'New':3})
    
    full["oSaleCondition"] = full.SaleCondition.map({'AdjLand':1, 'Abnorml':2, 'Alloca':2, 'Family':2, 'Normal':3, 'Partial':4})            
                
                        
                        
    
    return full

full2 = map_values(full2)
full3 = map_values(full3)
full4 = map_values(full4)

In [30]:

cols = full2.columns
for col in cols:
    na_value = 0
    full2[col].fillna(na_value, inplace = True)
    full3[col].fillna(na_value, inplace = True)
    full4[col].fillna(na_value, inplace = True)
# mapping 한것들 NA 값들도 넣어주기.


In [33]:
aa = full4.isnull().sum()
aa[aa>0]

Series([], dtype: int64)

In [37]:
for var in cols:
    if full4[var].dtype == int :
        full2[var] = full2[var].astype(float)
        full3[var] = full3[var].astype(float)
        full4[var] = full4[var].astype(float)

## 3. Model & Evaluation

### 3-1. Pipeline

In [9]:
# string -> one_hot_encoding
full2 = pd.get_dummies(full2)
full3 = pd.get_dummies(full3)
full4 = pd.get_dummies(full4)

In [10]:
full3.shape

(2919, 659)

In [44]:
# y 값
# label 분리 SalePrice 부분, 그리고 log scale까지
y = train['SalePrice']
y_log = np.log(y)
# x 값들 (columns, features, inputs)
n_train = train.shape[0]
x2 = full2[:n_train]
x3 = full3[:n_train]
x4 = full4[:n_train]

test_x2 = full2[n_train:]
test_x3 = full3[n_train:]
test_x4 = full4[n_train:]

### 3-2. Model & Eval

In [13]:
def rmse_cv(model,X,y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=5))
    return rmse

models = [LinearRegression(),
          Ridge(),
          Lasso(alpha=0.01,max_iter=10000),
          RandomForestRegressor(),
          GradientBoostingRegressor(),
          SVR(),
          LinearSVR(),
          ElasticNet(alpha=0.001,max_iter=10000),
          SGDRegressor(max_iter=1000,tol=1e-3),
          BayesianRidge(),
          KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5),
          ExtraTreesRegressor(),
          ]

# if you have xgboost package, put XGBRegressor() in models 

In [14]:
names = ["LR", "Ridge", "Lasso", "RF", "GBR", "SVR", "LinSVR", "Ela","SGD","Bay","Ker","Extra"]
for name, model in zip(names, models):
    score = rmse_cv(model, x2 , y_log)
    print("{}: {:.6f}, {:.4f}".format(name,score.mean(),score.std()))

LR: 1086474718.444026, 828495562.5575
Ridge: 0.154392, 0.0181
Lasso: 0.197213, 0.0110
RF: 0.151949, 0.0058
GBR: 0.130941, 0.0105
SVR: 0.156649, 0.0095
LinSVR: 0.170574, 0.0254
Ela: 0.134769, 0.0146
SGD: 0.235295, 0.0256
Bay: 0.145310, 0.0137
Ker: 0.154733, 0.0087
Extra: 0.152235, 0.0128


In [15]:
names = ["LR", "Ridge", "Lasso", "RF", "GBR", "SVR", "LinSVR", "Ela","SGD","Bay","Ker","Extra"]
for name, model in zip(names, models):
    score = rmse_cv(model, x3 , y_log)
    print("{}: {:.6f}, {:.4f}".format(name,score.mean(),score.std()))

LR: 1104457511.899359, 1869047709.1687
Ridge: 0.155605, 0.0216
Lasso: 0.160540, 0.0175
RF: 0.150792, 0.0111
GBR: 0.130811, 0.0103
SVR: 0.128753, 0.0104
LinSVR: 0.178378, 0.0244
Ela: 0.136310, 0.0224
SGD: 0.250555, 0.0309
Bay: 0.140253, 0.0197
Ker: 0.132936, 0.0154
Extra: 0.156889, 0.0140


In [16]:
names = ["LR", "Ridge", "Lasso", "RF", "GBR", "SVR", "LinSVR", "Ela","SGD","Bay","Ker","Extra"]
for name, model in zip(names, models):
    score = rmse_cv(model, x4 , y_log)
    print("{}: {:.6f}, {:.4f}".format(name,score.mean(),score.std()))

LR: 19812305.258886, 27840045.0994
Ridge: 0.155564, 0.0216
Lasso: 0.162120, 0.0175
RF: 0.148771, 0.0109
GBR: 0.131043, 0.0104
SVR: 0.212453, 0.0169
LinSVR: 0.296169, 0.0875
Ela: 0.136677, 0.0222
SGD: 317817739427472.125000, 181567248161772.5625
Bay: 0.140340, 0.0195
Ker: 0.344521, 0.3401
Extra: 0.150907, 0.0158


In [17]:
## Grid Search
class grid():
    def __init__(self,model):
        self.model = model
    
    def grid_get(self,X,y,param_grid):
        grid_search = GridSearchCV(self.model,param_grid,cv=5, scoring="neg_mean_squared_error")
        grid_search.fit(X,y)
        print(grid_search.best_params_, np.sqrt(-grid_search.best_score_))
        grid_search.cv_results_['mean_test_score'] = np.sqrt(-grid_search.cv_results_['mean_test_score'])
        print(pd.DataFrame(grid_search.cv_results_)[['params','mean_test_score','std_test_score']])

## Lasso

In [38]:
grid(Lasso()).grid_get(x2,y_log,{'alpha': [0.0004,0.0005,0.0007,0.0009],'max_iter':[10000]})
grid(Lasso()).grid_get(x3,y_log,{'alpha': [0.0004,0.0005,0.0007,0.0009],'max_iter':[10000]})
grid(Lasso()).grid_get(x4,y_log,{'alpha': [0.0004,0.0005,0.0007,0.0009],'max_iter':[10000]})

{'alpha': 0.0007, 'max_iter': 10000} 0.13482401197506705
                                 params  mean_test_score  std_test_score
0  {'alpha': 0.0004, 'max_iter': 10000}         0.136027        0.005014
1  {'alpha': 0.0005, 'max_iter': 10000}         0.134923        0.004507
2  {'alpha': 0.0007, 'max_iter': 10000}         0.134824        0.003929
3  {'alpha': 0.0009, 'max_iter': 10000}         0.135922        0.003578
{'alpha': 0.0007, 'max_iter': 10000} 0.13743098522108793
                                 params  mean_test_score  std_test_score
0  {'alpha': 0.0004, 'max_iter': 10000}         0.138768        0.006996
1  {'alpha': 0.0005, 'max_iter': 10000}         0.137967        0.006894
2  {'alpha': 0.0007, 'max_iter': 10000}         0.137431        0.006708
3  {'alpha': 0.0009, 'max_iter': 10000}         0.138097        0.006517
{'alpha': 0.0007, 'max_iter': 10000} 0.13763850030947286
                                 params  mean_test_score  std_test_score
0  {'alpha': 0.0004, 'max_

## Ridge

In [39]:
grid(Ridge()).grid_get(x2,y_log,{'alpha':[35,40,45,50,55,60,65,70,80,90]})
grid(Ridge()).grid_get(x3,y_log,{'alpha':[35,40,45,50,55,60,65,70,80,90]})
grid(Ridge()).grid_get(x4,y_log,{'alpha':[35,40,45,50,55,60,65,70,80,90]})

{'alpha': 35} 0.14891651284960963
          params  mean_test_score  std_test_score
0  {'alpha': 35}         0.148917        0.002732
1  {'alpha': 40}         0.149991        0.002707
2  {'alpha': 45}         0.151016        0.002691
3  {'alpha': 50}         0.151991        0.002681
4  {'alpha': 55}         0.152918        0.002676
5  {'alpha': 60}         0.153799        0.002674
6  {'alpha': 65}         0.154639        0.002676
7  {'alpha': 70}         0.155439        0.002679
8  {'alpha': 80}         0.156934        0.002691
9  {'alpha': 90}         0.158306        0.002707
{'alpha': 35} 0.1410153930180887
          params  mean_test_score  std_test_score
0  {'alpha': 35}         0.141015        0.005849
1  {'alpha': 40}         0.141025        0.005852
2  {'alpha': 45}         0.141077        0.005856
3  {'alpha': 50}         0.141157        0.005858
4  {'alpha': 55}         0.141256        0.005860
5  {'alpha': 60}         0.141368        0.005861
6  {'alpha': 65}         0.141488

## ElasticNet

In [40]:
grid(ElasticNet()).grid_get(x2,y_log,{'alpha':[0.0008,0.004,0.005],'l1_ratio':[0.08,0.1,0.3],'max_iter':[10000]})
grid(ElasticNet()).grid_get(x3,y_log,{'alpha':[0.0008,0.004,0.005],'l1_ratio':[0.08,0.1,0.3],'max_iter':[10000]})
grid(ElasticNet()).grid_get(x4,y_log,{'alpha':[0.0008,0.004,0.005],'l1_ratio':[0.08,0.1,0.3],'max_iter':[10000]})

{'alpha': 0.004, 'l1_ratio': 0.1, 'max_iter': 10000} 0.13796599583461197
                                              params  mean_test_score  \
0  {'alpha': 0.0008, 'l1_ratio': 0.08, 'max_iter'...         0.147658   
1  {'alpha': 0.0008, 'l1_ratio': 0.1, 'max_iter':...         0.146308   
2  {'alpha': 0.0008, 'l1_ratio': 0.3, 'max_iter':...         0.138468   
3  {'alpha': 0.004, 'l1_ratio': 0.08, 'max_iter':...         0.138163   
4  {'alpha': 0.004, 'l1_ratio': 0.1, 'max_iter': ...         0.137966   
5  {'alpha': 0.004, 'l1_ratio': 0.3, 'max_iter': ...         0.141983   
6  {'alpha': 0.005, 'l1_ratio': 0.08, 'max_iter':...         0.138502   
7  {'alpha': 0.005, 'l1_ratio': 0.1, 'max_iter': ...         0.138601   
8  {'alpha': 0.005, 'l1_ratio': 0.3, 'max_iter': ...         0.145411   

   std_test_score  
0        0.005417  
1        0.005331  
2        0.004820  
3        0.003373  
4        0.003232  
5        0.002677  
6        0.003093  
7        0.002956  
8        0.00258

## Kernel Ridge

In [41]:
param_grid={'alpha':[0.2,0.3,0.4], 'kernel':["polynomial"], 'degree':[3],'coef0':[0.8,1]}
grid(KernelRidge()).grid_get(x2,y_log,param_grid)
grid(KernelRidge()).grid_get(x3,y_log,param_grid)
grid(KernelRidge()).grid_get(x4,y_log,param_grid)

{'alpha': 0.2, 'coef0': 1, 'degree': 3, 'kernel': 'polynomial'} 0.1474327630602572
                                              params  mean_test_score  \
0  {'alpha': 0.2, 'coef0': 0.8, 'degree': 3, 'ker...         0.153305   
1  {'alpha': 0.2, 'coef0': 1, 'degree': 3, 'kerne...         0.147433   
2  {'alpha': 0.3, 'coef0': 0.8, 'degree': 3, 'ker...         0.159754   
3  {'alpha': 0.3, 'coef0': 1, 'degree': 3, 'kerne...         0.151845   
4  {'alpha': 0.4, 'coef0': 0.8, 'degree': 3, 'ker...         0.165367   
5  {'alpha': 0.4, 'coef0': 1, 'degree': 3, 'kerne...         0.155742   

   std_test_score  
0        0.002839  
1        0.002657  
2        0.003064  
3        0.002736  
4        0.003298  
5        0.002836  
{'alpha': 0.2, 'coef0': 1, 'degree': 3, 'kernel': 'polynomial'} 0.12805517789781531
                                              params  mean_test_score  \
0  {'alpha': 0.2, 'coef0': 0.8, 'degree': 3, 'ker...         0.134592   
1  {'alpha': 0.2, 'coef0': 1, 'degr

In [42]:
class AverageWeight(BaseEstimator, RegressorMixin):
    def __init__(self,mod,weight):
        self.mod = mod
        self.weight = weight
        
    def fit(self,X,y):
        self.models_ = [clone(x) for x in self.mod]
        for model in self.models_:
            model.fit(X,y)
        return self
    
    def predict(self,X):
        w = list()
        pred = np.array([model.predict(X) for model in self.models_])
        # for every data point, single model prediction times weight, then add them together
        for data in range(pred.shape[1]):
            single = [pred[model,data]*weight for model,weight in zip(range(pred.shape[0]),self.weight)]
            w.append(np.sum(single))
        return w

lasso = Lasso(alpha=0.0005,max_iter=10000)
ridge = Ridge(alpha=35)
svr = SVR(gamma= 0.0004,kernel='rbf',C=13,epsilon=0.009)
ker = KernelRidge(alpha=0.2 ,kernel='polynomial',degree=3 , coef0=0.8)
ela = ElasticNet(alpha=0.005,l1_ratio=0.08,max_iter=10000)
bay = BayesianRidge()

# assign weights based on their gridsearch score
w1 = 0.1
w2 = 0.0
w3 = 0.2
w4 = 0.5
w5 = 0.2
w6 = 0.0

weight_avg = AverageWeight(mod = [lasso,ridge,svr,ker,ela,bay],weight=[w1,w2,w3,w4,w5,w6])


score = rmse_cv(weight_avg,x3,y_log) #커널만 x3쓰기 pred에서 지금이 제일 낮은 item 찾음
print(score.mean())

0.12698408960784605


## stacking

## 3.3 Result - Submission

In [45]:
# 여러모델 평균
r1 = lasso.fit(x3,y_log)
r2 = ridge.fit(x3,y_log)
r3 = svr.fit(x3,y_log)
r4 = ker.fit(x3,y_log)
r5 = ela.fit(x3,y_log)
r6 = bay.fit(x3,y_log)

pred_r1 =np.exp(r1.predict(test_x3))
pred_r2 =np.exp(r2.predict(test_x3))
pred_r3 =np.exp(r3.predict(test_x3))
pred_r4 =np.exp(r4.predict(test_x3))
pred_r5 =np.exp(r5.predict(test_x3))
pred_r6 =np.exp(r6.predict(test_x3))

pred_final = w1 * pred_r1 + w2 * pred_r2 + w3 * pred_r3 + w4 * pred_r4 + w5 * pred_r5 + w6 * pred_r6
#pred_final =  w3 * pred_r3 + w4 * pred_r4 + w5 * pred_r5 


result=pd.DataFrame({'Id':test.Id, 'SalePrice':pred_final})
result.to_csv("submission6.csv",index=False)

In [47]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url = "https://user-images.githubusercontent.com/24144491/47170710-adfa5800-d341-11e8-8063-fe1312148e83.png")


In [92]:
Image(url = "https://user-images.githubusercontent.com/24144491/47187882-c3d34180-d370-11e8-9456-a86cc62615c0.PNG")

In [139]:
# categorical 변수 mapping 해서 numeric으로 넣어주니까 스코어는 더 낮아짐.
Image(url = "https://user-images.githubusercontent.com/24144491/47188599-ace21e80-d373-11e8-8799-7e0f289dd706.PNG")


In [140]:
# x2, x3 교차해서 결과내니 성능 더 낮아짐
Image(url = "https://user-images.githubusercontent.com/24144491/47188508-4c52e180-d373-11e8-8d9c-38f3c4b9d910.PNG")


In [141]:
## 추가로 해야할 것들, mapping 한 결과들 한 번 다 넣어보기. 그리고 lasso, 다른 결과들이 어떻게 내는지 확인해볼 것. 

In [48]:
Image(url = "https://user-images.githubusercontent.com/24144491/47236579-968d9e80-d417-11e8-8aab-9562d439fbe1.PNG")
# 0.01 개선